In [7]:
import pandas as pd
from mlp_ensemble import MlpEnsemble

#### **Getting training && validation data**

In [8]:
df_train = pd.read_csv('../databases/training.csv')
X_train, y_train = df_train.drop(['IND_BOM_1_1'], axis=1), df_train['IND_BOM_1_1']

In [9]:
df_valid = pd.read_csv('../databases/validation.csv')
X_val, y_val = df_valid.drop(['IND_BOM_1_1'], axis=1), df_valid['IND_BOM_1_1']

#### **MLP Ensemble**

In [10]:
models_params = [
    {
        'hidden_layer_units': 256,
        'activation': 'sigmoid',
        'hidden_layers': 1,
        'dropout_rate': 0.5,
        'output_activation': 'sigmoid',
        'alpha': 0.01,
        'optimizer': 'Adam',
        'loss_function': 'mse',
        'batch_size': 32,
        'max_iter': 2000
    },
    {
        'hidden_layer_units': 256,
        'activation': 'sigmoid',
        'hidden_layers': 1,
        'dropout_rate': 0.5,
        'output_activation': 'sigmoid',
        'alpha': 0.001,
        'optimizer': 'Adam',
        'loss_function': 'mse',
        'batch_size': 32,
        'max_iter': 5000
    },
    {
        'hidden_layer_units': 256,
        'activation': 'sigmoid',
        'hidden_layers': 1,
        'dropout_rate': 0.5,
        'output_activation': 'sigmoid',
        'alpha': 0.01,
        'optimizer': 'Adam',
        'loss_function': 'mse',
        'batch_size': 128,
        'max_iter': 2000
    },
]

In [11]:
input_dimension = X_train.shape[1]
ensemble_model = MlpEnsemble(models_params, input_dimension)

In [12]:
ensemble_model.fit(X_train, y_train, X_val, y_val)

Epoch 1/2000
5616/5616 [==============================] - 25s 4ms/step - loss: 0.2351 - accuracy: 0.6075 - val_loss: 0.2431 - val_accuracy: 0.5922
Epoch 2/2000
5616/5616 [==============================] - 24s 4ms/step - loss: 0.2339 - accuracy: 0.6091 - val_loss: 0.2186 - val_accuracy: 0.6411
Epoch 3/2000
5616/5616 [==============================] - 24s 4ms/step - loss: 0.2321 - accuracy: 0.6137 - val_loss: 0.2540 - val_accuracy: 0.5491
Epoch 4/2000
5616/5616 [==============================] - 24s 4ms/step - loss: 0.2315 - accuracy: 0.6145 - val_loss: 0.2294 - val_accuracy: 0.5900
Epoch 5/2000
5616/5616 [==============================] - 24s 4ms/step - loss: 0.2314 - accuracy: 0.6160 - val_loss: 0.2138 - val_accuracy: 0.6620
Epoch 6/2000
5616/5616 [==============================] - 24s 4ms/step - loss: 0.2309 - accuracy: 0.6175 - val_loss: 0.2368 - val_accuracy: 0.6005
Epoch 7/2000
5616/5616 [==============================] - 25s 4ms/step - loss: 0.2302 - accuracy: 0.6201 - val_loss: 0

In [13]:
ensemble_model.get_score(X_val, y_val)

0.6144664616342898

In [14]:
ensemble_model.save_model()

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...layers\dropout
......vars
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-04-06 22:31:38         2231
metadata.json                                  2023-04-06 22:31:38           64
variables.h5                                   2023-04-06 22:31:38       531800
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...layers\dropout
......vars
...metrics\mean
......vars
.........0
.